### Estructura del Circuito

Este circuito se encarga de calcular:

$
c = (a^2 + b^2) \mod p
$

donde:

- **Entradas Privadas:**

  - **`a` y `b`:** Valores numéricos cuyos cuadrados se suman.
- **Entrada Pública:**
  - **`p`:** El módulo para la operación, que es público.
- **Salida (Publica):**
  - **`c`:** El resultado final, es decir, el resto de la división de $(a^2 + b^2)$ entre $p$.

#### Flujo de Operaciones y Cálculos Intermedios

1. **Cálculo de Cuadrados:**  

   Se calcula $a^2$ y $b^2$.

2. **Suma de los Cuadrados:**  

   Se suma $a^2 + b^2$ para obtener `suma`.

3. **Cálculo del Módulo:**  

   Se utiliza el operador de asignación insegura `<--` para asignar a `resto` el valor de `suma % p` porque la operación módulo no se puede representar directamente como restricción cuadrática.  
   Además, se calcula el cociente `q` de la división de `suma` entre `p`.

4. **Imposición de la Relación:**  

   Se añade la restricción:

   $
   \text{suma} = q \times p + \text{resto}
   $

   para garantizar que el valor asignado a `resto` sea correcto.

5. **Verificación del Rango:**  

   Se utiliza un componente (`LessThan(252)`) para asegurarse de que:

   $
   \text{resto} < p
   $

6. **Salida:**  

   La salida `c` se asigna como el valor de `resto`.


---

###  Proceso de Generación de Pruebas

##### 1. Compilación del Circuito

Se utiliza **Circom** para transformar el código del circuito en artefactos que son necesarios para la siguiente etapa.  
Esto genera:

- Un archivo **R1CS** (el sistema de restricciones)
- Un archivo **WASM** (código WebAssembly que se usará para calcular el testigo)
- Un archivo **.sym** (información simbólica para depuración)

##### 2. Generación del Testigo

Con el archivo **WASM** y un archivo de entrada que define los valores para las señales del circuito se calcula el **testigo**.  
El testigo es el conjunto de valores internos que satisfacen todas las restricciones definidas en el circuito.

##### 3. Configuración de la Ceremonia de Confianza (Powers of Tau)

En este paso se generan **parámetros universales** que servirán para el esquema de pruebas **Groth16**.

- Primero se crea un archivo inicial de **Powers of Tau**
- Se añade **contribución de entropía** para reforzar la seguridad
- Luego se prepara el archivo para la **fase 2**, de modo que esté listo para usarse en el *setup* del circuito

##### 4. Setup de Groth16

Se utiliza el archivo **R1CS** junto con el archivo de parámetros (preparado en la fase anterior) para crear la clave inicial del sistema de prueba, conocida como **zkey**.

Pasos:

- Añadir una **contribución de entropía**
- Obtener la **clave final**
- Exportar la **clave de verificación**, que es pública y se utiliza para validar la prueba sin revelar datos privados

##### 5. Generación de la Prueba (Prove)

Con la clave final (**zkey**) y el **testigo generado**, se crea la prueba criptográfica propiamente dicha.

Esto genera dos archivos:

- `proof.json`: contiene la prueba
- `public.json`: contiene las señales públicas


---

### Proceso de Verificación

##### 1. Lectura de Archivos

Se leen los siguientes archivos generados durante el proceso de pruebas:

- `verification_key.json`: Exportada durante el setup.
- `proof.json`: La prueba generada.
- `public.json`: Las señales públicas.


##### 2. Ejecución de la Verificación

Se utiliza la función `snarkjs.groth16.verify` para comprobar que la prueba cumple con las restricciones del circuito.


##### En el Script (build.sh)

En el script `build.sh` incluye una verificación ejecutando:
```
snarkjs groth16 verify verification_key.json public.json proof.json
```

##### En Node.js:

Esto se realiza en el script `verify.js` que:

- Carga los archivos (usando `fs`).
- Llama a `snarkjs.groth16.verify(verificationKey, publicSignals, proof)`.
- Imprime `"Prueba válida"` si la función devuelve `true`, o `"Prueba inválida"` si devuelve `false`.

##### En el navegador:

Esto se realiza con un HTML que:

- Carga `snarkjs` desde un CDN.
- Utiliza `fetch` para obtener `verification_key.json`, `proof.json` y `public.json`.
- Llama a `snarkjs.groth16.verify` y muestra el resultado en la interfaz.


---

### Ejemplos de uso con valores concretos

##### Ejemplo 1

**Archivo de Entrada (`input.json`)**

```json
{
  "a": 3,
  "b": 4,
  "p": 5
}
```

**Resultado:**

- La salida `c` es `0`.
- La entrada pública `p` es `5`.

**Archivo `public.json`:**

```json
[ "0", "5" ]
```

##### Ejemplo 2

**Archivo de Entrada (`input.json`)**

```json
{
  "a": 2,
  "b": 3,
  "p": 7
}
```

**Resultado:**

- La salida `c` es `6`.
- La entrada pública `p` es `7`.

**Archivo `public.json`:**

```json
[ "6", "7" ]
```
